In [13]:
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel

app = FastAPI()
# Allow CORS for all origins
origins = [
    "http://localhost:5173",  # for local dev
]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,  # or ["*"] temporarily
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)


In [11]:
from fastapi import WebSocket, WebSocketDisconnect
import asyncio
import json
import websockets

COINBASE_WS_URL = "wss://ws-feed.exchange.coinbase.com"


In [14]:
@app.websocket("/ws/orderbook")
async def orderbook_stream(websocket: WebSocket):
    await websocket.accept()
    try:
        async with websockets.connect(COINBASE_WS_URL) as ws:
            await ws.send(json.dumps({
                "type": "subscribe",
                "channels": [{"name": "level2", "product_ids": ["BTC-USD"]}]
            }))
            async for message in ws:
                data = json.loads(message)
                if data.get("type") == "snapshot":
                    if data.get("type") in ["l2update", "snapshot"]:
                        update = {
                            "bids": data.get("bids", []),
                            "asks": data.get("asks", []),
                            "type": data.get("type")
                        }
                        await websocket.send_json(update)
    except WebSocketDisconnect:
        pass
    except Exception as e:
        await websocket.close(code=1000, reason=str(e))
        print(f"WebSocket error: {e}")

In [20]:
import asyncio
import websockets

async def test_ws():
    uri = "ws://localhost:8000/ws/orderbook"
    async with websockets.connect(uri) as websocket:
        print("✅ Connected")
        for _ in range(5):  # just get 5 messages to avoid infinite loop
            msg = await websocket.recv()
            print("📨", msg)

await test_ws()

OSError: Multiple exceptions: [Errno 61] Connect call failed ('::1', 8000, 0, 0), [Errno 61] Connect call failed ('127.0.0.1', 8000)